In [1]:
from elasticsearch import Elasticsearch, helpers
from rank_bm25 import BM25Okapi
import pandas as pd
from datasets import load_from_disk, Dataset
from transformers import AutoTokenizer
from tqdm import tqdm
import pickle
import numpy as np
from pprint import pprint
import re

In [2]:
!service elasticsearch start
# !service elasticsearch stop

 * Starting Elasticsearch Server
sysctl: setting key "vm.max_map_count": Read-only file system
   ...done.


In [4]:
!ps -ef | grep elastic

elastic+ 14617     1 99 07:19 pts/1    00:01:14 /usr/share/elasticsearch/jdk/bin/java -Xshare:auto -Des.networkaddress.cache.ttl=60 -Des.networkaddress.cache.negative.ttl=10 -XX:+AlwaysPreTouch -Xss1m -Djava.awt.headless=true -Dfile.encoding=UTF-8 -Djna.nosys=true -XX:-OmitStackTraceInFastThrow -XX:+ShowCodeDetailsInExceptionMessages -Dio.netty.noUnsafe=true -Dio.netty.noKeySetOptimization=true -Dio.netty.recycler.maxCapacityPerThread=0 -Dio.netty.allocator.numDirectArenas=0 -Dlog4j.shutdownHookEnabled=false -Dlog4j2.disable.jmx=true -Djava.locale.providers=SPI,COMPAT --add-opens=java.base/java.io=ALL-UNNAMED -XX:+UseG1GC -Djava.io.tmpdir=/tmp/elasticsearch-6790927117882694420 -XX:+HeapDumpOnOutOfMemoryError -XX:HeapDumpPath=/var/lib/elasticsearch -XX:ErrorFile=/var/log/elasticsearch/hs_err_pid%p.log -Xlog:gc*,gc+age=trace,safepoint:file=/var/log/elasticsearch/gc.log:utctime,pid,tags:filecount=32,filesize=64m -Xms31744m -Xmx31744m -XX:MaxDirectMemorySize=16642998272 -XX:InitiatingHeapO

In [3]:
wiki_dataset = pd.read_json('/opt/ml/data/preprocess_wiki.json',orient='index') # 전처리된 위키
wiki_dataset = wiki_dataset.drop_duplicates(['text','title'],ignore_index=True) # 중복 제거

train_dataset = load_from_disk('/opt/ml/data/train_dataset/train').to_pandas()
valid_dataset = load_from_disk('/opt/ml/data/train_dataset/validation').to_pandas()

In [4]:
with open('/opt/ml/data/wiki_context_id_pair.bin','rb') as f:
    wiki_context_id_pair = pickle.load(f)

with open('/opt/ml/data/wiki_id_context_pair.bin','rb') as f:
    wiki_id_context_pair = pickle.load(f)

In [5]:
#wiki_corpus = list(set([example_wiki for example_wiki in wiki_dataset['text']]))
train_context = train_dataset['context']
valid_context = valid_dataset['context']
train_query = train_dataset['question']
valid_query = valid_dataset['question']

In [6]:
wiki_corpus = list(wiki_context_id_pair.keys())
tokenizer = AutoTokenizer.from_pretrained('klue/bert-base')
tokenized_corpus = [tokenizer.tokenize(context) for context in wiki_corpus]
bm25 = BM25Okapi(tokenized_corpus)

Token indices sequence length is longer than the specified maximum sequence length for this model (1131 > 512). Running this sequence through the model will result in indexing errors


In [8]:
INDEX_NAME = "wiki_index"
INDEX_SETTINGS = {
    "settings": {
        "index": {
            "analysis": {
                "analyzer": {
                    "korean": {
                        "type": "custom",
                        "tokenizer": "nori_tokenizer",
                        "filter": ["shingle"],
                    }
                }
            }
        }
    },
    "mappings": {
        "properties": {
            "content": {
                "type": "text",
                "analyzer": "korean",
                "search_analyzer": "korean",
            },
            "title": {
                "type": "text",
                "analyzer": "korean",
                "search_analyzer": "korean",
            },
        }
    },
}


In [9]:
docs = [
    {
        '_index' : INDEX_NAME,
        '_id' : wiki_dataset.iloc[i]['document_id'],
        'title' : wiki_dataset.iloc[i]['title'],
        'content' : wiki_dataset.iloc[i]['text']
    }
    for i in range(wiki_dataset.shape[0])
]

In [10]:
#!ps -ef | grep elastic

In [11]:
try:
    es.transport.close()
except:
    pass
#config = {'host':'localhost', 'port':2293}
es = Elasticsearch()

In [12]:
es.info()

/opt/conda/lib/python3.8/site-packages/elasticsearch/connection/base.py:209: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


{'name': '89aa4310917b',
 'cluster_name': 'elasticsearch',
 'cluster_uuid': 'tWsEBuE7ReuuGa-pZkywtw',
 'version': {'number': '7.15.1',
  'build_flavor': 'default',
  'build_type': 'deb',
  'build_hash': '83c34f456ae29d60e94d886e455e6a3409bba9ed',
  'build_date': '2021-10-07T21:56:19.031608185Z',
  'build_snapshot': False,
  'lucene_version': '8.9.0',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

In [13]:
if es.indices.exists(INDEX_NAME):
    es.indices.delete(index=INDEX_NAME)
es.indices.create(index=INDEX_NAME, body=INDEX_SETTINGS)

<ipython-input-13-b31c6028f9a8>:1: DeprecationWarning: Using positional arguments for APIs is deprecated and will be disabled in 8.0.0. Instead use only keyword arguments for all APIs. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  if es.indices.exists(INDEX_NAME):
<ipython-input-13-b31c6028f9a8>:3: DeprecationWarning: The 'body' parameter is deprecated for the 'create' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.indices.create(index=INDEX_NAME, body=INDEX_SETTINGS)


{'acknowledged': True, 'shards_acknowledged': True, 'index': 'wiki_index'}

In [14]:
import time
start_time = time.time()
try:
    response = helpers.bulk(es, docs)
    print ("\nRESPONSE:", response)
except Exception as e:
    print("\nERROR:", e)
    
end_time = time.time()

print(end_time-start_time)


RESPONSE: (56044, [])
103.60818409919739


In [17]:
try:
    res = es.search(index=INDEX_NAME, q=train_query[0], size=10)
except:
    res = es.search(index=INDEX_NAME, q=train_query[0])

In [16]:
# pprint(train_query[0])
# pprint(es.indices.analyze(index=INDEX_NAME, body={'analyzer' : 'korean', 'text' : train_query[0]}))
# res['hits']['hits'][0]['_id']

In [19]:
# reader_data
train_data_with_elastic = {}
train_data_with_elastic_score = {}

error_questions = []
top_k_num = 200

for i in tqdm(range(len(train_query))):
    try:
        res = es.search(index=INDEX_NAME, q=train_query[i], size=top_k_num)
        hits = res['hits']['hits']
        
        top_k_list = []
        top_k_score = []
        
        for hit in hits:

            #wiki_id = hit['_id']
            ctx = hit['_source']['content']
            score = hit['_score']
            
            top_k_list.append(wiki_context_id_pair[ctx])
            top_k_score.append(score)
        
        train_data_with_elastic[train_query[i]] = top_k_list
        train_data_with_elastic_score[train_query[i]] = top_k_score
    
    except Exception as e:
        error_questions.append([e, train_query[i]])
        
        top_n_id_text = []
        top_n_text = bm25.get_top_n(tokenizer.tokenize(train_query[i]), wiki_corpus, n=top_k_num)
        
        for ctx in top_n_text:
            #wiki_id = wiki_dataset['document_id'][wiki_dataset['text'] == ctx]
            top_n_id_text.append(wiki_context_id_pair[ctx])
        #train_data_with_elastic[train_query[i]] = bm25.get_top_n(tokenizer.tokenize(train_query[i]), wiki_corpus, n=top_k_num)
        
        train_data_with_elastic[train_query[i]] = top_n_id_text
        temp_score = bm25.get_scores(tokenizer.tokenize(train_query[i])).tolist()
        temp_score.sort(reverse=True)
        train_data_with_elastic_score[train_query[i]] = temp_score[:top_k_num]
    
    

100%|██████████| 3952/3952 [05:30<00:00, 11.97it/s]


In [18]:
# reader_data
valid_data_with_elastic = {}
valid_data_with_elastic_score = {}
error_questions = []
top_k_num = 200
for i in tqdm(range(len(valid_query))):
    try:
        res = es.search(index=INDEX_NAME, q=valid_query[i], size=top_k_num)
        hits = res['hits']['hits']
        
        top_k_list = []
        top_k_score = []
        
        for hit in hits:

            wiki_id = hit['_id']
            ctx = hit['_source']['content']
            score = hit['_score']
            
            top_k_list.append(wiki_context_id_pair[ctx])
            top_k_score.append(score)
        
        valid_data_with_elastic[valid_query[i]] = top_k_list
        valid_data_with_elastic_score[valid_query[i]] = top_k_score
    
    except Exception as e:
        error_questions.append([e, valid_query[i]])
        
        top_n_id_text = []
        top_n_text = bm25.get_top_n(tokenizer.tokenize(valid_query[i]), wiki_corpus, n=top_k_num)
        
        for ctx in top_n_text:
            #wiki_id = wiki_dataset['document_id'][wiki_dataset['text'] == ctx]
            top_n_id_text.append(wiki_context_id_pair[ctx])
        #train_data_with_elastic[valid_query[i]] = bm25.get_top_n(tokenizer.tokenize(valid_query[i]), wiki_corpus, n=top_k_num)
        
        valid_data_with_elastic[valid_query[i]] = top_n_id_text
        temp_score = bm25.get_scores(tokenizer.tokenize(valid_query[i])).tolist()
        temp_score.sort(reverse=True)
        valid_data_with_elastic_score[valid_query[i]] = temp_score[:top_k_num]

100%|██████████| 240/240 [00:21<00:00, 11.25it/s]


In [19]:
# with open('/opt/ml/data/elastic_train_200.bin', "wb") as file:
#     pickle.dump(train_data_with_elastic, file)
# with open('/opt/ml/data/elastic_train_score_200.bin', "wb") as file:
#     pickle.dump(train_data_with_elastic_score, file)
with open('/opt/ml/data/elastic_valid_200.bin', "wb") as file:
    pickle.dump(valid_data_with_elastic, file)
with open('/opt/ml/data/elastic_valid_score_200.bin', "wb") as file:
    pickle.dump(valid_data_with_elastic_score, file)

In [20]:
query_idx = 0
rank = 5

print(valid_data_with_elastic[valid_query[query_idx]][rank])
print(valid_data_with_elastic_score[valid_query[query_idx]][rank])
print(wiki_id_context_pair[valid_data_with_elastic[valid_query[query_idx]][rank]])


46605
21.818851
(연구사례1) 내적 통제위치를 가진 사람이 외적 통제위치를 가진 사람보다 조직후원인식이 더 높은 것으로 나타났다. (연구사례2) 통제의 위치와 조직후원인식이 긍정적인 상관관계를 가지는 것으로 나타났다. (연구사례3) 내재적 보상을 통한 자율성을 증가시키는 것은 조직이 종업원에 대해 관심을 가지고, 인정한다는 것을 느끼게 한다. 자율성이 증가하게 되면, 종업원은 업무와 그 결과에 대해 강한 개인적 책임감을 가지기 때문에, 내재적 동기를 유발시킨다. 이는 조직후원인식의 형성에 긍정적인 영향을 미치는 것으로 나타났다. 조직후원인식의 선행변수에 대한 기존의 연구들은 두 개의 선행변수로 요약할 수있다. 하나는 최고경영자, 상사, 동료들을 신뢰하고, 지원한다는 인식이며, 다른 하나는 조직이 투자하는 훈련과 개발, 교육을 위한 휴직과 같은 인적자원관행과 승진, 임금인상과 보너스 등의 재정적 보상과 같은 조직의 인정이다. 이 외에도 조직, 신뢰, 복지 그리고 공정성 등이 연구되고 있으며, 동시에 조직의 도덕성과 리더십에 의해서도 영향을 받는다는 연구가 있다. 이러한 연구들을 종합해보면, 조직후원인식에 영향을 주는 선행변수는 공정성 변수로 보상에 대한 분배공정성과 절차공정성, 조직의 성과평가에 대한 절차공정성이 많았다


In [23]:
test_dataset = load_from_disk('/opt/ml/data/test_dataset/validation').to_pandas()
test_query = test_dataset['question']

In [24]:
test_data_with_elastic = {}
test_data_with_elastic_score = {}
error_test_questions = []
top_k_num = 200

for i in tqdm(range(len(test_query))):
    try:
        res = es.search(index=INDEX_NAME, q=test_query[i], size=top_k_num)
        hits = res['hits']['hits']
        
        top_k_list = []
        top_k_score = []
        
        for hit in hits:

            wiki_id = hit['_id']
            ctx = hit['_source']['content']
            score = hit['_score']
            
            top_k_list.append(wiki_context_id_pair[ctx])
            top_k_score.append(score)
        
        test_data_with_elastic[test_query[i]] = top_k_list
        test_data_with_elastic_score[test_query[i]] = top_k_score
    
    except Exception as e:
        error_questions.append([e, test_query[i]])
        
        top_n_id_text = []
        top_n_text = bm25.get_top_n(tokenizer.tokenize(test_query[i]), wiki_corpus, n=top_k_num)
        
        for ctx in top_n_text:
            #wiki_id = wiki_dataset['document_id'][wiki_dataset['text'] == ctx]
            top_n_id_text.append(wiki_context_id_pair[ctx])
        #train_data_with_elastic[test_query[i]] = bm25.get_top_n(tokenizer.tokenize(test_query[i]), wiki_corpus, n=top_k_num)
        
        test_data_with_elastic[test_query[i]] = top_n_id_text
        temp_score = bm25.get_scores(tokenizer.tokenize(test_query[i])).tolist()
        temp_score.sort(reverse=True)
        test_data_with_elastic_score[test_query[i]] = temp_score[:top_k_num]

100%|██████████| 600/600 [00:50<00:00, 11.96it/s]


In [25]:
with open('/opt/ml/data/elastic_test_200.bin', "wb") as file:
    pickle.dump(test_data_with_elastic, file)
with open('/opt/ml/data/elastic_test_score_200.bin', "wb") as file:
    pickle.dump(test_data_with_elastic_score, file)

In [26]:
# with open('/opt/ml/data/elastic_train_100.bin', "rb") as file:
#     train_data_with_elastic = pickle.load(file)
with open('/opt/ml/data/elastic_valid_200.bin', "rb") as file:
    valid_data_with_elastic = pickle.load(file)

In [28]:
for i in range(len(valid_query)):
    print(valid_query[i])
    print(valid_data_with_elastic[valid_query[i]][0])
    print(wiki_id_context_pair[17913])
    break

처음으로 부실 경영인에 대한 보상 선고를 받은 회사는?
17913
기업을 운영하는 데 있어서는 우선 인적 주체가 누구이냐가 문제되는바, 이는 기업의 형태 문제와 관련된다. 상법과 민법은 개인기업의 형태를 택할 때의 '상인'에 관한 개념과, 공동기업의 형태를 택할 때의 '영업자'와 '익명조합원' 또는 '공동사업자'에 관한 개념과, 법인기업의 형태를 택할 때의 '법인'의 개념을 규제한다. 그리고 법인기업(회사기업)의 설립에 관하여는 정관(定款)의 작성과 설립등기(設立登記)에 관하여 자세한 규제를 하고 있다. 회사라고 하더라도 주식회사(株式會社)이냐, 유한회사(有限會社)이냐, 합명회사(合名會社)이냐 혹은 합자회사(合資會社)이냐에 따라서 각각 다른 내용의 규제를 하고 있다. 기업의 규모에 관하여도 법적 규제가 있다. 예컨대 유한회사에 있어서는 자본금을 천만원 이상으로 규제하고 (상법 546조 1항), 금융기관의 자본금은 250억원 이상으로 규제한다(은행법 16조 1항). 이와 같은 예는 증권거래법에도 있고, 기타의 특별법에도 산재한다. 일단 설립된 주식회사가 증자(增資)를 통하여 규모를 확대하는 경우와 감자(減資)를 통하여 규모를 축소시키려는 경우에 상법 기타 특별법의 규제가 가해진다. 기업이 조직변형을 하는 경우에도 법적 규제를 받아야 한다. 이를테면 주식회사는 유한회사로, 그리고 유한회사는 주식회사로 조직변경할 수 있다. 또한 합명회사는 합자회사로, 그리고 합자회사(合資會社)는 합명회사(合名會社)로 조직변경할 수가 있다. 그러나 인적 회사(합명회사·합자회사)가 물적 회사(주식회사·유한회사)로 바뀐다거나 물적 회사가 인적 회사로 바뀔 수는 없다. 개인기업이 법인조직으로 되는 경우에는 상법상 새로운 법인설립의 절차를 밟아야 한다. 회사가 합병을 하는 경우에도 엄격한 법적 규제를 받게 된다. 외국의 경우에는 독점금지법(獨占禁止法)과 같은 법규로서 기업의 합병을 억제하기도 한다. 자본의 자유화를 위해서도 법적 규제가 나타난다. 기업의 체질개선을 위해서는 상법상 주식의 자

In [87]:
res = es.search(index=INDEX_NAME, q=valid_query[0], size=100)

In [88]:
res

{'took': 48,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 10000, 'relation': 'gte'},
  'max_score': 26.632221,
  'hits': [{'_index': 'wiki_index',
    '_type': '_doc',
    '_id': '17913',
    '_score': 26.632221,
    '_source': {'title': '기업 경영에 대한 법규',
     'content': "기업을 운영하는 데 있어서는 우선 인적 주체가 누구이냐가 문제되는바, 이는 기업의 형태 문제와 관련된다. 상법과 민법은 개인기업의 형태를 택할 때의 '상인'에 관한 개념과, 공동기업의 형태를 택할 때의 '영업자'와 '익명조합원' 또는 '공동사업자'에 관한 개념과, 법인기업의 형태를 택할 때의 '법인'의 개념을 규제한다. 그리고 법인기업(회사기업)의 설립에 관하여는 정관(定款)의 작성과 설립등기(設立登記)에 관하여 자세한 규제를 하고 있다. 회사라고 하더라도 주식회사(株式會社)이냐, 유한회사(有限會社)이냐, 합명회사(合名會社)이냐 혹은 합자회사(合資會社)이냐에 따라서 각각 다른 내용의 규제를 하고 있다. 기업의 규모에 관하여도 법적 규제가 있다. 예컨대 유한회사에 있어서는 자본금을 천만원 이상으로 규제하고 (상법 546조 1항), 금융기관의 자본금은 250억원 이상으로 규제한다(은행법 16조 1항). 이와 같은 예는 증권거래법에도 있고, 기타의 특별법에도 산재한다. 일단 설립된 주식회사가 증자(增資)를 통하여 규모를 확대하는 경우와 감자(減資)를 통하여 규모를 축소시키려는 경우에 상법 기타 특별법의 규제가 가해진다. 기업이 조직변형을 하는 경우에도 법적 규제를 받아야 한다. 이를테면 주식회사는 유한회사로, 그리고 유한회사는 주식회사로 조직변경할 수 있다.

In [60]:
temp_df = wiki_dataset.drop_duplicates(['text','title'])
temp_df[temp_df['title'] == '전효숙']

,text,corpus_source,url,domain,title,author,html,document_id
4473,"순천여자고등학교 졸업, 1973년 이화여자대학교를 졸업하고 1975년 제17회 사법...",위키피디아,TODO,NaN,전효숙,NaN,NaN,4473
4474,2006년 8월 16일 노무현 대통령으로부터 헌법재판소장으로 지명되어 여성 최초의 ...,위키피디아,TODO,NaN,전효숙,NaN,NaN,4474


In [ ]:
# INDEX_NAME = "wiki_index"
# INDEX_SETTINGS = {
#   "settings" : {
#     "index":{
#       "analysis":{
#         "analyzer":{
#           "korean":{
#             "type":"custom",
#             "tokenizer":"nori_tokenizer",
#             "filter": [ "nori_posfilter","nori_readingform","shingle"],
#           }
#         },
#         "filter":{
#           "nori_posfilter":{
#             "type":"nori_part_of_speech",
#             "stoptags":[
#                 #"E", # 어미
#                 #"IC", # 감탄사
#                 #"J", # 조사
#                 #"MAJ" # 접속부사
#                 #"MAG", # 일반부사
#                 #"MM", #관형사
#                 "NA", #알려지지 않은
#                 #"NR", #수사
#                 #"SC",#구분기호
#                 #"SE",#생략
#                 #"SF",#마침표, 물음표, 느낌표
#                 "SH",#한자
#                 "SL",#외국어
#                 #"SN",#숫자
#                 "SP",#띄어쓰기
#                 "SSC",#닫는괄호
#                 "SSO",#여는괄호
#                 "SY",#기타기호
#                 "UNA",#알수 없는
#                 "UNKNOWN",#알수없는
#                 #"VA",#형용사
#                 #"VCN",#부정지정사
#                 #"VCP",#긍정 지정사
#                 "VSV",#알수없는
#                 #"VV",#동사
#                 #"VX",#보조동사 또는 형용사
#                 #"XPN",#접두사
#                 #"XR",#어근
#                 #"XSA",#형용사 접미사
#                 #"XSN",#명사 접미사
#                 #"XSV"#동사 접미사
#             ]
#           }
#         }
#         }
#       },
#   },
#   "mappings": {
#       "properties" : {
#         "content" : {
#           "type" : "text",
#           "analyzer": "korean",
#           "search_analyzer": "korean"
#         },
#         "title" : {
#           "type" : "text",
#           "analyzer": "korean",
#           "search_analyzer": "korean"
#         }
#       }
#   }
# }

In [ ]:
# gq negative sampling

# retrieval_data_with_elastic = {}
# retrieval_data_with_elastic_score = {}
# error_questions = []
# top_k_num = 101
# for i in tqdm(range(len(retrieval_query))):
#     try:

#         res = es.search(index=INDEX_NAME, q=retrieval_query[i], size=top_k_num)
#         hits = res['hits']['hits']

#         top_k_list = []
#         top_k_score = []

#         for hit in hits:

#             wiki_id = hit['_id']
#             #ctx = hit['_source']['content']
#             score = hit['_score']

#             top_k_list.append(int(wiki_id))
#             top_k_score.append(score)

#         if len(top_k_list) == top_k_num:
#             retrieval_data_with_elastic[retrieval_query[i]] = top_k_list
#             retrieval_data_with_elastic_score[retrieval_query[i]] = top_k_score
#         else:
#             print('under_top_k')
#             print(retrieval_query[i])

#     except Exception as e:
#         print('elastic_error')
#         print(retrieval_query[i])

#         error_questions.append([e, retrieval_query[i]])

#         top_n_id_text = []
#         top_n_text = bm25.get_top_n(tokenizer.tokenize(retrieval_query[i]), wiki_corpus, n=top_k_num)
#         if len(top_n_text) == top_k_num:
#             for ctx in top_n_text:

#                 wiki_id = wiki_dataset['document_id'][wiki_dataset['text'] == ctx]
#                 top_n_id_text.append(int(wiki_id))

#             #train_data_with_elastic[retrieval_query[i]] = bm25.get_top_n(tokenizer.tokenize(retrieval_query[i]), wiki_corpus, n=top_k_num)
#             retrieval_data_with_elastic[retrieval_query[i]] = top_n_id_text
#             temp_score = bm25.get_scores(tokenizer.tokenize(retrieval_query[i])).tolist()
#             temp_score.sort(reverse=True)
#             retrieval_data_with_elastic_score[retrieval_query[i]] = temp_score[:top_k_num]